In [70]:
import time, copy
import numpy as np

def get_func(func):
  if func == 1:
    # optimum = 0
    D = 30
    feasible_bounds = [-100, 100]
    init_bounds = [50, 100]
    return D, feasible_bounds, init_bounds
  elif func == 2:
    # optimum = 0
    D = 30
    feasible_bounds = [-100, 100]
    init_bounds = [50, 100]
    return D, feasible_bounds, init_bounds
  elif func == 3:
    # optimum = 0
    D = 30
    feasible_bounds = [-30, 30]
    init_bounds = [15, 30]
    return D, feasible_bounds, init_bounds
  elif func == 4:
    # optimum = 420.9687
    D = 30
    feasible_bounds = [-500, 500]
    init_bounds = [-500, -250]
    return D, feasible_bounds, init_bounds
  elif func == 5:
    # optimum = 0
    D = 30
    feasible_bounds = [-5.12, 5.12]
    init_bounds = [2.56, 5.12]
    return D, feasible_bounds, init_bounds

def f1_sphere(X):
  result = np.zeros(len(X))
  for i in range(len(X)):
    for j in range(len(X[i])):
      result[i] += X[i][j] ** 2
  return result

def f2_schwefel12(X):
  result = np.zeros(len(X))
  for i in range(len(X)):
    for j in range(len(X[i])):
      temp = 0
      for k in range(j):
        temp += X[i][k]
      result[i] += temp ** 2
  return result

def f3_generalized_rosenbrock(X):
  result = np.zeros(len(X))
  for i in range(len(X)):
    for j in range(len(X[i])-1):
      result[i] += 100 * (X[i][j+1] - X[i][j]**2)**2 + (X[i][j]-1)**2
  return result

def f4_generalized_schwefel26(X):
  result = np.zeros(len(X))
  D = len(X[0])
  for i in range(len(X)):
    for j in range(len(X[i])):
      result[i] += X[i][j] * np.sin(np.sqrt(np.abs(X[i][j])))
    result[i] = 418.9829 * D - result[i]
  return result

def f5_generalized_rastrigin(X):
  result = np.zeros(len(X))
  for i in range(len(X)):
    for j in range(len(X[i])):
      result[i] += X[i][j] ** 2 - 10 * np.cos(2 * np.pi * X[i][j]) + 10
  return result

def get_fitness(X, func):
  if func == 1:
    return f1_sphere(X)
  elif func == 2:
    return f2_schwefel12(X)
  elif func == 3:
    return f3_generalized_rosenbrock(X)
  elif func == 4:
    return f4_generalized_schwefel26(X)
  elif func == 5:
    return f5_generalized_rastrigin(X)


def selection(population, fitness, k=3):
  parents = np.empty((population.shape[0], population.shape[1]))
  for parent_num in range(population.shape[0]):
    selection_ix = np.random.randint(len(population))
    for ix in np.random.randint(0, len(population), k-1):
      if fitness[ix] < fitness[selection_ix]:
        selection_ix = ix
    parents[parent_num, :] = population[selection_ix, :]
  return parents

def crossover(parents, pc, bound):  #Ricky: 交配
  dim_size = parents.shape[1]
  offspring = copy.copy(parents)
  for i in range(int(parents.shape[0] / 2)):
    if np.random.random() > pc:
      x, y = 2*i, 2*i+1
      beta = np.random.randint(11)/10
      k = np.random.randint(dim_size) # crossover point
      offspring[x][k] = parents[x][k] + beta * (parents[y][k] - parents[x][k])
      offspring[y][k] = bound[0] + beta * (bound[1] - bound[0])
      offspring[x][k+1:] = parents[y][k+1:]
      offspring[y][k+1:] = parents[x][k+1:]
  return offspring

def mutation(offspring, pm): #Ricky: 突變 
  d_list = np.array([i for i in range(offspring.shape[1])])
  for i in range(offspring.shape[0]):
    if np.random.random() < pm: #Ricky: 如果符合突變條件
      mutation_point = np.random.choice(d_list, 2, replace=False)
      beta = np.random.randint(11)/10
      x = (1 - beta) * offspring[i][mutation_point[0]]\
        + beta * offspring[i][mutation_point[1]]
      y = beta * offspring[i][mutation_point[0]]\
        + (1 - beta) * offspring[i][mutation_point[1]]
      offspring[i][mutation_point[0]] = x
      offspring[i][mutation_point[1]] = y
  return offspring


start_time = time.time()
np.random.seed(7)

functions = [1, 2, 3, 4, 5]

result = []
for func in functions:
  A_time = time.time()
  dimension_size, feasible_bounds, init_bounds = get_func(func)
  population_size = 50 #50 #600
  iteration_size = 1800 #2000 #300
  pc = 0.8
  pm = 0.05
  best_solution, best_fitness = [], []
  aa = 0

  population = np.random.uniform(
        low = init_bounds[0], 
        high = init_bounds[1], 
        size = (population_size, dimension_size))
  fitness = np.zeros(population_size)
  fitness = get_fitness(population, func)

  for iter in range(iteration_size):
    parents = selection(population, fitness)
    offspring_crossover = crossover(parents, pc, feasible_bounds)
    offspring_mutation = mutation(offspring_crossover, pm)
    population[:] = offspring_mutation[:]
    fitness = get_fitness(population, func)
    if(np.min(fitness) < 1):
      break

  aa = np.min(fitness)
  if(np.min(fitness) > 100):
    dimension_size, feasible_bounds, init_bounds = get_func(func)
    population_size = 900
    iteration_size = 350
    pc = 0.8
    pm = 0.05
    best_solution, best_fitness = [], []

    population = np.random.uniform(
          low = init_bounds[0], 
          high = init_bounds[1], 
          size = (population_size, dimension_size))
    fitness = np.zeros(population_size)
    fitness = get_fitness(population, func)
    for iter in range(iteration_size):
      parents = selection(population, fitness)
      offspring_crossover = crossover(parents, pc, feasible_bounds)
      offspring_mutation = mutation(offspring_crossover, pm)
      population[:] = offspring_mutation[:]
      fitness = get_fitness(population, func)
      if(np.min(fitness) < 1):
        break
      elif(np.min(fitness) - aa == 0):
        break
  result.append(np.min(fitness))
print(result)
print(np.mean(result))
end_time = time.time()
print("Spend %0.4f sec(s)" % (end_time - start_time))

[0.9506241558224915, 0.9443350628223202, 29.716781461805258, 1614.1439912684727, 0.0]
329.15114638978457
Spend 84.0962 sec(s)
